In [60]:
# Ciselne algoritmy - DU2 - Josef Sykora
#
# Pollard-(p-1) for factorization

# version 1 of the algorithm from lecture
def pollard_p_minus_1_v1(N, B, a):
    if a is None:  # allow choice of a
        a = randint(1, N - 1)
        a = R(a)
    
    d = gcd(a, N)
    if d > 1:
        return d, N / ZZ(d), "if_1"

    P = Primes()
    p = P.first()
    while p <= B:
        c_i = floor(log(B, p))
        e_i = p^c_i
        a = a^e_i
        d = gcd(a - 1, N)
        if d > 1:
            if d < N:
                return d, N / ZZ(d), "if_2"
            else:
                return "Fail_1"
        p = P.next(p)
    return "Fail_2"

# version 2 of the algorithm from lecture
def pollard_p_minus_1(N, B, a, e_prime_B):
    if a is None:  # allow choice of a
        a = randint(1, N - 1)
        a = R(a)
    
    d = gcd(a, N)
    if d > 1:
        return d, N / ZZ(d), "if_1"
    
    if e_prime_B is None:  # for precomputation
        e_prime_B = get_exponent(B, 1)
    
    a = a^e_prime_B
    
    d = gcd(ZZ(a) - 1, N)  # we need to interpret a as integer so the gcd works over Z
    if d > 1:
        if d < N:
            return d, N / ZZ(d), "if_2"
        else:
            return "Fail_1"
    
    stop = floor(log(B, 2))
    for i in range(stop):
        a = a^2
        d = gcd(ZZ(a) - 1, N)
        if d > 1:
            if d < N:
                return d, N / ZZ(d), "if_3"
            else:
                return "Fail_2"
    
    return  "Fail_3"

# calculate e_B
def get_exponent(B, start):
    result = 1
    P = Primes()
    p = P.unrank(start)  # indexed from 0
    log_p = floor(log(B,p))
    result *= p^(log_p)
    
    while True:
        p = P.next(p)
        if p > B:
            break
        log_p = floor(log(B, p))
        result *= p^(log_p)
    
    return result

# calculate smallest B that can yield a factorization non-trivially
# (i.e. exists nontrivial (gcd(a, N)=1) a: algorithm finds factorization with this choice)
def smallest_B(N, start_B, max_B):
    PP = Primes()
    B = PP.next(start_B - 1)
    while B <= max_B:
        precompute_e = get_exponent(B, 1)
        for i in range(1, N):
            res = pollard_p_minus_1(N, B, R(i), precompute_e)
            if type(res) != type("Fail"):
                if res[2] != "if_1":  # remove to allow a divides N trivially
                    print("Smallest possible B:", B)
                    return B
        B = PP.next(B)
    return "B not in given range"

# try factoring a number using Pollard-(p-1) multiple times
def factor_num(N, B, max_trys):
    for i in range(max_trys):
        res = pollard_p_minus_1(N, B, None, None)
        if type(res) != type("Fail"):
            print("Found factorization of", N, "=", res[0], "x", res[1], "in", i + 1, "trys.")
            break
    return res


# driver code:

# assigned: 3589
# 3827, 3901, 4819, 3869, 3649, 3569, 3713, 3589, 3599, 3337, 3763, 3811, 4183, 4187, 3977
my_N = 3589
my_B = 20
R = IntegerModRing(my_N)


factor_num(my_N, 2, 40)
smallest_B(my_N, 2, 50)
print("Calculations ended.")

Found factorization of 3589 = 97 x 37 in 16 trys.
Smallest possible B: 2
Calculations ended.
